# MIDITok - Generating!

## 1. Install & Import necessary libraries

In [1]:
# !pip install miditok
# !pip install pygame
# !pip install note-seq

In [5]:
from transformers import GPT2LMHeadModel
from miditok import MMM, TokenizerConfig, REMI
import torch
import pygame
import datetime
import note_seq
import os

🎹 MIDI 재생을 위한 play_midi 함수 정의

In [3]:
def play_midi(midi_file):
    pygame.init()
    pygame.mixer.init()

    try:
        pygame.mixer.music.load(midi_file)
        pygame.mixer.music.play()

        while pygame.mixer.music.get_busy():
            pygame.time.Clock().tick(10)
    except KeyboardInterrupt:
        pygame.mixer.music.stop()
        pygame.quit()

## 2. 학습한 모델 불러오기

기 학습 가중치 불러오기

In [4]:
model = GPT2LMHeadModel.from_pretrained("model_none/checkpoint-5800") # 본인의 모델 경로로 바꿔주기!
# model = GPT2LMHeadModel.from_pretrained("models/checkpoint-3700").cuda() # 서버에서 추론시 .cuda() 붙여주기

학습한 모델과 동일한 설정의 토크나이저 사용하기

In [7]:
TOKENIZER_PARAMS = {
    "pitch_range": (0, 127),
    "num_velocities": 127,
    "use_chords": True,
    "use_tempos": True,
    "use_programs": True,
    "num_tempos": 211,  # number of tempo bins
    "tempo_range": (40, 250),  # (min, max)
}
config = TokenizerConfig(**TOKENIZER_PARAMS)
TOKENIZER_NAME = REMI
tokenizer = TOKENIZER_NAME(config)

## 3. 생성하기

### 🔄 초기화 셀
- 처음 입력할 토큰을 지정해주고, tensor로 바꾸어 generated_ids 변수에 할당합니다
- 기존 생성된 결과를 초기화하고 새로 생성하고 싶다면, 아래 셀을 실행하면 됩니다

In [14]:
# 생성 토큰 초기화
# initial_token = "BOS_None" # 처음에는 문장 시작을 알리는 "BOS_None"만 넣어줍니다
# generated_ids = torch.tensor([[tokenizer[initial_token]]])

next_tokens = "BOS_None Bar_None Position_0 Tempo_120.0"
generated_ids = torch.tensor([[tokenizer[token] for token in next_tokens.split(" ")]])

# print(generated_ids)
midi_tokens = generated_ids[0]
# iteration number와 현재 timestamp 초기화
iteration_number = 0
ts = datetime.datetime.now().strftime("%y%m%d_%H%M%S") # 출력 MIDI 파일명에 사용

### 🪄 생성 셀!
- 아래 셀을 실행하여 생성하세요.
- 여러 번 실행하면 실행 할 때마다 트랙이 추가됩니다.

In [15]:
# Iteration 몇 번 돌았는지 기록
iteration_number += 1
print(f"current iteration : {iteration_number}")

while len(midi_tokens)<4000:
    print(len(midi_tokens))
    # Encode the conditioning tokens.
    input_ids = generated_ids # 서버에서 추론할 경우 .cuda() 추가
    point = 1
    if len(input_ids[0]) > 700:
        for idx, token in enumerate(input_ids[0]):
            if token == tokenizer["Bar_None"]:
                
                if len(input_ids[0])-idx <700:
                    input_ids = input_ids[:, idx:]
                    break
                # point = 1

    save_point = len(input_ids[0])

    # Generate more tokens.
    eos_token_id = tokenizer["Bar_None"]  # "Bar_None" 토큰이 나올 때까지 생성 => iteration당 한 트랙씩 생성하는 원리
    # eos_token_id =tokenizer["EOS_None"]
    temperature = 0.8 # Temperature를 높이면 생성 결과가 더욱 randomize 되는 것 같습니다. 0~1 사이 값으로 자유롭게 시도해보세요!


    generated_ids = model.generate(
        input_ids,
        max_length=1024,
        do_sample=True,
        temperature=temperature,
        eos_token_id=eos_token_id,
    ).cuda()
    generated_ids =generated_ids.cpu()


    midi_tokens = torch.concat([midi_tokens, generated_ids[0][save_point:]])
# MidiTok 라이브러리의 tokens_to_midi 기능 이용하여 미디 데이터로 변환
midi = tokenizer.tokens_to_midi(midi_tokens)
midi.dump_midi('test_gen.mid') # 임시 MIDI 파일 생성

note_sequence = note_seq.midi_file_to_note_sequence('test_gen.mid') # 시각화를 위해 note_sequence로 변환
print(note_sequence.instrument_infos)


try:
    note_seq.plot_sequence(note_sequence, show_figure=True) # 미디 파일 시각화
except:
    pass


# MIDI 파일 재생
play_midi('test_gen.mid')

current iteration : 1
4
변경 전 4
변경 후 4
129
변경 전 129
변경 후 129
258
변경 전 258
변경 후 258
442
변경 전 442
변경 후 442
629
변경 전 629
변경 후 629
1024
변경 전 1024
수정 됨
수정 됨
수정 됨
수정 됨
변경 후 583
1026
변경 전 585
변경 후 585
1233
변경 전 792
수정 됨
수정 됨
변경 후 605
1466
변경 전 838
수정 됨
수정 됨
변경 후 441
1701
변경 전 676
변경 후 676
2049
변경 전 1024
수정 됨
수정 됨
수정 됨
변경 후 584
2130
변경 전 665
변경 후 665
2304
변경 전 839
수정 됨
수정 됨
변경 후 604
2479
변경 전 779
수정 됨
수정 됨
변경 후 350
2630
변경 전 501
변경 후 501
2748
변경 전 619
변경 후 619
2874
변경 전 745
수정 됨
수정 됨
변경 후 571
2999
변경 전 696
변경 후 696
3132
변경 전 829
수정 됨
수정 됨
변경 후 654
3295
변경 전 817
수정 됨
수정 됨
변경 후 666
3481
변경 전 852
수정 됨
수정 됨
수정 됨
변경 후 608
3659
변경 전 786
수정 됨
수정 됨
변경 후 661
[name: "Acoustic Grand Piano"
, instrument: 1
name: "Acoustic Bass"
, instrument: 2
name: "Drums"
, instrument: 3
name: "Fretless Bass"
]


/tmp/ipykernel_1348945/1296252928.py:3: RuntimeWarning: use mixer: libgthread-2.0.so.0: cannot open shared object file: No such file or directory
(ImportError: libgthread-2.0.so.0: cannot open shared object file: No such file or directory)
  pygame.mixer.init()


NotImplementedError: mixer module not available (ImportError: libgthread-2.0.so.0: cannot open shared object file: No such file or directory)

In [ ]:
# # Iteration 몇 번 돌았는지 기록
# iteration_number += 1
# print(f"current iteration : {iteration_number}")

# # Encode the conditioning tokens.
# input_ids = generated_ids # 서버에서 추론할 경우 .cuda() 추가

# # Generate more tokens.
# eos_token_id = tokenizer["Track_End"]  # "Track_End" 토큰이 나올 때까지 생성 => iteration당 한 트랙씩 생성하는 원리
# temperature = 0.8 # Temperature를 높이면 생성 결과가 더욱 randomize 되는 것 같습니다. 0~1 사이 값으로 자유롭게 시도해보세요!
# generated_ids = model.generate(
#     input_ids,
#     max_length=1024,
#     do_sample=True,
#     temperature=temperature,
#     eos_token_id=eos_token_id,
# ).cpu()

# # MidiTok 라이브러리의 tokens_to_midi 기능 이용하여 미디 데이터로 변환
# midi = tokenizer.tokens_to_midi(generated_ids[0])
# midi.dump_midi('test_gen.mid') # 임시 MIDI 파일 생성

# # MIDI 파일 시각화
# note_sequence = note_seq.midi_file_to_note_sequence('test_gen.mid') # 시각화를 위해 note_sequence로 변환
# # 아랫줄 주석 풀고 실행하면 에러가 날텐데, 에러가 뜨는 notebook_utils.py 파일 링크로 들어가셔서 fig.plot_width 를 fig.width로, fig.plot_height를 fig.height로 고치면 돌아감
# # note_seq.plot_sequence(note_sequence, show_figure=True) 

# # 확인용 출력문
# # print(midi) # 미디 파일 정보
# # print(generated_ids) # 생성된 토큰 아이디

# # MIDI 파일 재생
# play_midi('test_gen.mid')

🏃 트랙을 더 추가하고 싶으면 다시 위로 올라가서 [🪄 생성 셀] 을 실행하세요~!

### 💾 저장 셀
- 현재 iteration의 생성 결과를 outputs 폴더에 output-timestamp-iteration 형태의 파일명으로 저장합니다.

In [ ]:
# 'outputs' 폴더 없으면 생성
output_folder = 'outputs'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 'outputs' 폴더 내에 MIDI 파일 저장
output_file_path = os.path.join(output_folder, f'output-at-{ts}-iter{iteration_number}.mid')
midi.dump_midi(output_file_path)

### (Optional) 아래 두 셀은 생성 토큰 내용을 자연어 형태로 확인하고 싶을 때만 실행하세요!

확인용 vocab dict 생성

In [ ]:
vocab_dict = {}
for k,v in tokenizer.vocab.items():
    vocab_dict[v] = k

생성된 id 자연어 형태 토큰으로 직접 확인

In [ ]:
generated_tokens = []
for g in generated_ids[0].tolist():
    generated_tokens.append(vocab_dict[g])
generated_tokens